In [1]:
from mpl_toolkits.mplot3d import Axes3D
from rtree import index
import xgboost as xgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import math

import keras
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

Using TensorFlow backend.


In [2]:
%store -r mc_data

In [3]:
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        xlayer_model = load_model('./models/xlayer_model.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [4]:
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        track_classifier = load_model('./models/track_classifier.h5')

In [4]:
###### Data columns
i = 0
for val in mc_data.columns:
    print(i," ",val)
    i = i + 1

0   MCParticleID
1   MC_Hit_X
2   MC_Hit_Y
3   MC_Hit_Z
4   PrHit_LHCbID
5   PrHit_Xat0
6   PrHit_Zat0
7   PrHit_isX
8   PrHit_planeCode
9   PrHit_w2
10   PrHit_yMax
11   PrHit_yMin
12   PrHit_zone
13   event
14   hasT
15   isInUT
16   isInVelo
17   isLong
18   isLong_fromB
19   isLong_fromB_more5
20   isLong_more5


In [5]:
mc_hits = mc_data.values

In [6]:
%store -r zLays
%store -r max_x
%store -r min_x
%store -r max_y
%store -r min_y
%store -r events

In [7]:
print(events)

[15837 15838 15839 15840 15841 15842 15843 15844 15845 15846 15847 15848
 15849 15850 15851 15852 15853 15854 15855 15856 15857 15858 15859 15860
 15861 15862 15863 15864 15865 15866 15867 15868 15869 15870 15871 15872
 15873 15874 15875 15876 15877 15878 15879 15880 15881 15882 15883 15884
 15885 15886 15887 15888 15889 15890 15891 15892 15893 15894 15895 15896
 15897 15898 15899 15900 15901 15902 15903 15904 15905 15906 15907 15908
 15909 15910 15911 15912 15913 15914 15915 15916 15917 15918 15919 15920
 15921 15922 15923 15924 15925 15926 15927 15928 15929 15930 15931 15932
 15933 15934 15935 15936 15937 15938 15939 15940 15941 17909 17910 17911
 17912 17913 17914 17915 17916 17917 17918 17919 17920 17921 17922 17923
 17924 17925 17926 17927 17928 17929 17930 17931 17932 17933 17934 17935
 17936 17937 17938 17939 17940 17941 17942 17943 17944 17945 17946 17947
 17948 17949 17950 17951 17952 17953 17954 17955 17956 17957 17958 17959
 17960 17961 17962 17963 17964 17965 17966 17967 17

In [8]:
event_threshold = 15840

In [61]:
hits_total = mc_hits[mc_hits[:,13] > event_threshold]
print("Total hits with threshold %d" % (len(hits_total)))
event = 0
idx = {}
actual = 0
for hit in hits_total:
    if (not hit[13] in idx):
        p = index.Property()
        p.dimension = 3
        idx[hit[13]] = index.Index(properties=p, interleaved=False)
    
    if (len(hit[5]) > 1) and (hit[0] != 11) and (hit[0] != -11):
        for i in range(0,len(hit[5])):
            idx[hit[13]].insert(int(hit[4][i]), (float(hit[5][i]), float(hit[5][i]), hit[8][i], hit[8][i], hit[12][i], hit[12][i]), [float(hit[5][i]), int(hit[4][i])])

Total hits with threshold 199144


KeyboardInterrupt: 

In [9]:
def predictHeight(x0, x3, x_stereo, stereo_layer, first_layer, last_layer):
    
    z_stereo = zLays[stereo_layer]
    
    if (stereo_layer % 2 != 0):
        angle = 5
    else:
        angle = -5
        
    m = float(zLays[last_layer] - zLays[first_layer]) / float(x3 - x0)
    n = zLays[last_layer] - (m * x3)
    
    x_pred = (z_stereo - n) / m 
    beta = (x_pred - x_stereo) / (math.radians(angle) * z_stereo)
    return math.tan(beta) * z_stereo

In [18]:
#
# Test the NN model by using events not feeded as train 
#

def trackReconstruction(hits_total, index_tree):

    # Variables used for calculating metrics
    total_found = 0
    total = 0
    mean_size = np.zeros(3)

    # Variable defining the maximum number of hits gathered from the R-trees when using the predicted value from the NN
    max_hits_gathered = 12

    # Range in which valid hits are look up in the first R-tree
    long_range = 300

    X_classifier = []
    Y_classifier = []

    # For each particle of a given event
    for particle in hits_total:

        # Sometimes there are no PrHits for a given particle, so skip these ones
        if (len(particle[5]) > 1) and (particle[0] != 11) and (particle[0] != -11):

            found_num = 0            
            X_final = {}
            Zone_final = {}
            actual = -1

            # Extract all PrHits
            for i in range(0, len(particle[5])):
                if (actual != particle[8][i]):
                    X_final[particle[8][i]] = []
                    X_final[particle[8][i]].append(particle[5][i])
                    Zone_final[particle[8][i]] = particle[12][i]
                    actual = particle[8][i]
                    found_num += 1
                else:
                    X_final[actual].append(particle[5][i])

            # Check if there are minimum 12 hits found, as for the test, I am only working with complete tracks
            if (found_num < 12):
                continue

            # List used to iterate over valid height found later on
            valid_y = {}

            # For each station
            for station in range(0,3):

                valid_y[station] = []

                # Compute the layer number based on station
                x0_layer = 0
                if (station == 1):
                    x0_layer = 4
                elif station == 2:
                    x0_layer = 8

                u_layer = x0_layer + 1
                v_layer = x0_layer + 2
                x1_layer = x0_layer + 3

                # Select both the first X position and UP/DOWN based on this
                first_hit = X_final[x0_layer][0]
                zone = Zone_final[x0_layer]

                # Predict all 3 following points based on the first hit selected
                predicted = (xlayer_model.predict(np.array([first_hit, x0_layer]).reshape(1,-1)))[0]
                mean_predicted = ((first_hit + predicted[0] + predicted[1] + predicted[2]) / 4)

                # Recolect all nearest hits from each layer using R-trees based on the X-position of the mean predicted
                if (x1_layer in X_final):
                    found = list(index_tree.nearest((mean_predicted - long_range, mean_predicted + long_range, x1_layer, x1_layer, zone, zone), objects='raw'))
                    found_layer3 = sorted(found, key=lambda x: np.abs(x[0]- mean_predicted))[:max_hits_gathered]
                    arr_layer3 = np.unique(np.array(found_layer3), axis=0)

                if (v_layer in X_final):
                    found = list(index_tree.nearest((mean_predicted - long_range, mean_predicted + long_range, v_layer, v_layer, zone, zone), objects='raw'))
                    found_layer2 = sorted(found, key=lambda x: np.abs(x[0] - mean_predicted))[:max_hits_gathered]
                    arr_layer2 = np.unique(np.array(found_layer2), axis=0)

                if (u_layer in X_final):
                    found = list(index_tree.nearest((mean_predicted - long_range, mean_predicted + long_range, u_layer, u_layer, zone, zone), objects='raw'))
                    found_layer1 = sorted(found, key=lambda x: np.abs(x[0] - mean_predicted))[:max_hits_gathered]
                    arr_layer1 = np.unique(np.array(found_layer1), axis=0)

                # Now, for each hit found in the last layer, as it is the one with most precision, 
                # tracks are searched
                for h3 in arr_layer3:

                    # Sometimes a hit in the last layer is equal to the first one
                    if (h3[0] == first_hit):
                        continue

                    for h2 in arr_layer2:

                        # Predict height for each hit selected in the V layer
                        pred_h2 = predictHeight(first_hit, h3[0], h2[0], v_layer, x0_layer, x1_layer)
                        if (pred_h2 > 0) and (pred_h2 > max_y) or (pred_h2 < 0) and (pred_h2 < min_y):
                            continue

                        # And for each hit in the V layer, we look up at hits in the U layer with the same height
                        for h1 in arr_layer1:

                            # Predict height for hit in the U layer
                            pred_h1 = predictHeight(first_hit, h3[0], h1[0], u_layer, x0_layer, x1_layer)
                            if (pred_h1 > 0) and (pred_h1 > max_y) or (pred_h1 < 0) and (pred_h1 < min_y):
                                continue

                            # Compare then, if the combination of U+V+X makes sense based on the height.
                            # As they usually have a slight difference between them, so only if the height is similar
                            # it is considered a valid track
                            if np.abs(pred_h2 - pred_h1) < 15:
                                # Insert into a new R-tree to enable search by height
                                # Store the valid height for later iterate over it
                                valid_y[station].append([h1[0], h2[0], h3[0], pred_h2, (h1[1],h2[1],h3[1])])

            valid_tracks = []
            for val in valid_y[0]:
                if (len(valid_y[1]) > 0 and len(valid_y[2]) > 0):
                    second_sort = sorted(valid_y[1], key = lambda p: np.abs(p[3] - val[3]))
                    if (np.abs(second_sort[0][3] - val[3]) < 100):
                        third_sort = sorted(valid_y[2], key = lambda p: np.abs(p[3] - second_sort[0][3]))
                        if np.abs(third_sort[0][3] - second_sort[0][3]) < 100:
                            valid_tracks.append([val, second_sort[0], third_sort[0]])

            repeated = 0
            len_tracks = len(valid_tracks)
            for track in valid_tracks:
                found = 0
                for station in range(0,3):
                    x0_layer = 0
                    if (station == 1):
                        x0_layer = 4
                    elif station == 2:
                        x0_layer = 8

                    u_layer = x0_layer + 1
                    v_layer = x0_layer + 2
                    x1_layer = x0_layer + 3

                    pos = 0
                    zone = 0
                    if (track[station][3] > 0):
                        zone = 1

                    for id in track[station][4]:
                        if (id in particle[4]):
                            found += 1
                            index_tree.delete(int(id), (track[station][pos], track[station][pos], x0_layer + pos + 1, x0_layer + pos + 1, zone, zone))
                        pos += 1

                if (found > 6):
                    X_classifier.append(track[0][:4] + track[1][:4] + track[2][:4])
                    Y_classifier.append(1)
                    len_tracks -= 1
                    repeated += 1
                else:
                    X_classifier.append(track[0][:4] + track[1][:4] + track[2][:4])
                    Y_classifier.append(0)

            if (repeated > 0):
                total_found += 1

            mean_size[0] += (len_tracks)
            total += 1
    #print("\nMean Total Found: %f" % (total_found/total))
    #print("\nMean Size: %f" % (mean_size[0]/total))
    return X_classifier, Y_classifier, total_found/total, mean_size[0]/total
    

In [19]:
mean = size = done = 0
test_events = events[3:]

X_classifier = []
Y_classifier = []

p = index.Property()
p.dimension = 3
    
for event in test_events:
    particles_test_event = mc_hits[mc_hits[:,13] == event]
    # print("Event length %d " % len(particles_test_event))
    idx = {}
    mean_val = size_val = 0
    index_tree = index.Index(properties=p, interleaved=False)
    for hit in particles_test_event:
        if (len(hit[5]) > 1) and (hit[0] != 11) and (hit[0] != -11):
            for i in range(0,len(hit[5])):
                index_tree.insert(int(hit[4][i]), (float(hit[5][i]), float(hit[5][i]), hit[8][i], hit[8][i], hit[12][i], hit[12][i]), [float(hit[5][i]), int(hit[4][i])])

    X, Y, mean_val, size_val = trackReconstruction(particles_test_event, index_tree)
    
    X_classifier += X
    Y_classifier += Y
    mean += mean_val
    size += size_val
    done += 1
    print("Event %d/%d \t Acc: %f Size %f Proportion %f" % (done,len(test_events), mean_val, size_val, Y.count(0)/(Y.count(1) + 1)))
    
print("\nFinal Result:")
print("\t X_Size : %f" % (len(X_classifier)))
print("\t Y_Size: %f" % (len(Y_classifier)))
print("\t Y count 0 : %f" % (Y_classifier.count(0)))
print("\t Y count 1: %f" % (Y_classifier.count(1)))
print("\t Mean: %f" % (mean/len(test_events)))
print("\t Size: %f" % (size/len(test_events)))

Event 1/926 	 Acc: 0.695652 Size 0.231884 Proportion 0.219178
Event 2/926 	 Acc: 0.754098 Size 1.270492 Proportion 0.824468
Event 3/926 	 Acc: 0.804348 Size 0.065217 Proportion 0.054545
Event 4/926 	 Acc: 0.684211 Size 0.368421 Proportion 0.341463
Event 5/926 	 Acc: 0.782609 Size 0.652174 Proportion 0.545455
Event 6/926 	 Acc: 0.913793 Size 0.155172 Proportion 0.081818
Event 7/926 	 Acc: 0.766667 Size 0.833333 Proportion 0.602410
Event 8/926 	 Acc: 0.777778 Size 0.000000 Proportion 0.000000
Event 9/926 	 Acc: 0.750000 Size 0.045455 Proportion 0.044444
Event 10/926 	 Acc: 0.677419 Size 0.096774 Proportion 0.107143
Event 11/926 	 Acc: 0.814815 Size 0.000000 Proportion 0.000000
Event 12/926 	 Acc: 0.830189 Size 0.245283 Proportion 0.185714
Event 13/926 	 Acc: 0.893617 Size 0.042553 Proportion 0.031250
Event 14/926 	 Acc: 0.816901 Size 0.042254 Proportion 0.034884
Event 15/926 	 Acc: 0.588652 Size 2.056738 Proportion 1.959459
Event 16/926 	 Acc: 0.703704 Size 0.000000 Proportion 0.000000
E

Event 131/926 	 Acc: 0.846154 Size 0.051282 Proportion 0.043478
Event 132/926 	 Acc: 1.000000 Size 0.100000 Proportion 0.083333
Event 133/926 	 Acc: 0.738095 Size 0.119048 Proportion 0.128205
Event 134/926 	 Acc: 0.842105 Size 0.210526 Proportion 0.145455
Event 135/926 	 Acc: 0.743590 Size 0.153846 Proportion 0.171429
Event 136/926 	 Acc: 0.616071 Size 5.026786 Proportion 3.585987
Event 137/926 	 Acc: 0.828125 Size 0.656250 Proportion 0.411765
Event 138/926 	 Acc: 0.684685 Size 1.945946 Proportion 1.430464
Event 139/926 	 Acc: 0.746479 Size 0.591549 Proportion 0.512195
Event 140/926 	 Acc: 0.666667 Size 2.992908 Proportion 2.232804
Event 141/926 	 Acc: 0.743902 Size 0.829268 Proportion 0.635514
Event 142/926 	 Acc: 0.867925 Size 0.150943 Proportion 0.103896
Event 143/926 	 Acc: 0.904762 Size 0.301587 Proportion 0.234568
Event 144/926 	 Acc: 0.746269 Size 0.462687 Proportion 0.360465
Event 145/926 	 Acc: 0.660377 Size 1.367925 Proportion 1.188525
Event 146/926 	 Acc: 0.859649 Size 0.122

Event 260/926 	 Acc: 0.806818 Size 1.090909 Proportion 0.842105
Event 261/926 	 Acc: 0.743363 Size 1.150442 Proportion 0.855263
Event 262/926 	 Acc: 0.627907 Size 0.186047 Proportion 0.200000
Event 263/926 	 Acc: 0.648855 Size 2.496183 Proportion 1.767568
Event 264/926 	 Acc: 0.891892 Size 0.027027 Proportion 0.019608
Event 265/926 	 Acc: 0.700000 Size 0.414286 Proportion 0.333333
Event 266/926 	 Acc: 0.611842 Size 2.447368 Proportion 2.125714
Event 267/926 	 Acc: 0.688889 Size 0.511111 Proportion 0.442308
Event 268/926 	 Acc: 0.622517 Size 2.814570 Proportion 2.125000
Event 269/926 	 Acc: 0.731481 Size 1.083333 Proportion 0.921260
Event 270/926 	 Acc: 0.669492 Size 2.491525 Proportion 1.860759
Event 271/926 	 Acc: 0.744681 Size 0.276596 Proportion 0.224138
Event 272/926 	 Acc: 0.657407 Size 2.111111 Proportion 1.727273
Event 273/926 	 Acc: 0.813559 Size 0.050847 Proportion 0.036585
Event 274/926 	 Acc: 0.797297 Size 0.364865 Proportion 0.278351
Event 275/926 	 Acc: 0.777778 Size 0.037

Event 389/926 	 Acc: 0.888889 Size 0.000000 Proportion 0.000000
Event 390/926 	 Acc: 0.755556 Size 0.133333 Proportion 0.117647
Event 391/926 	 Acc: 0.628099 Size 1.438017 Proportion 1.348837
Event 392/926 	 Acc: 0.892857 Size 0.000000 Proportion 0.000000
Event 393/926 	 Acc: 0.933333 Size 0.100000 Proportion 0.061224
Event 394/926 	 Acc: 0.575000 Size 2.712500 Proportion 2.451977
Event 395/926 	 Acc: 0.746032 Size 0.301587 Proportion 0.237500
Event 396/926 	 Acc: 0.716981 Size 0.641509 Proportion 0.515152
Event 397/926 	 Acc: 0.810811 Size 0.472973 Proportion 0.372340
Event 398/926 	 Acc: 0.697248 Size 2.651376 Proportion 2.064286
Event 399/926 	 Acc: 0.903226 Size 0.000000 Proportion 0.000000
Event 400/926 	 Acc: 0.593220 Size 1.135593 Proportion 1.313725
Event 401/926 	 Acc: 0.833333 Size 0.055556 Proportion 0.040000
Event 402/926 	 Acc: 0.655738 Size 1.860656 Proportion 1.554795
Event 403/926 	 Acc: 0.714286 Size 0.342857 Proportion 0.272727
Event 404/926 	 Acc: 0.836364 Size 0.127

Event 518/926 	 Acc: 0.854167 Size 0.145833 Proportion 0.107692
Event 519/926 	 Acc: 0.725490 Size 1.241830 Proportion 1.010638
Event 520/926 	 Acc: 0.752381 Size 1.609524 Proportion 1.126667
Event 521/926 	 Acc: 0.704225 Size 2.732394 Proportion 1.969543
Event 522/926 	 Acc: 0.658824 Size 1.094118 Proportion 0.885714
Event 523/926 	 Acc: 0.655914 Size 0.720430 Proportion 0.650485
Event 524/926 	 Acc: 0.723810 Size 1.276190 Proportion 1.000000
Event 525/926 	 Acc: 0.800000 Size 0.428571 Proportion 0.375000
Event 526/926 	 Acc: 0.659091 Size 0.295455 Proportion 0.250000
Event 527/926 	 Acc: 0.724490 Size 0.622449 Proportion 0.476562
Event 528/926 	 Acc: 0.760870 Size 0.173913 Proportion 0.150943
Event 529/926 	 Acc: 0.709677 Size 0.752688 Proportion 0.619469
Event 530/926 	 Acc: 0.818182 Size 0.136364 Proportion 0.120000
Event 531/926 	 Acc: 0.702703 Size 1.063063 Proportion 0.975207
Event 532/926 	 Acc: 0.780822 Size 0.643836 Proportion 0.479592
Event 533/926 	 Acc: 0.669811 Size 1.584

Event 647/926 	 Acc: 0.641221 Size 1.885496 Proportion 1.524691
Event 648/926 	 Acc: 0.600000 Size 0.200000 Proportion 0.183673
Event 649/926 	 Acc: 0.671875 Size 0.531250 Proportion 0.586207
Event 650/926 	 Acc: 0.618421 Size 0.289474 Proportion 0.209524
Event 651/926 	 Acc: 0.700000 Size 0.740000 Proportion 0.649123
Event 652/926 	 Acc: 0.888889 Size 0.305556 Proportion 0.207547
Event 653/926 	 Acc: 0.683333 Size 0.283333 Proportion 0.253731
Event 654/926 	 Acc: 0.603774 Size 2.830189 Proportion 2.173913
Event 655/926 	 Acc: 0.604938 Size 1.493827 Proportion 1.531646
Event 656/926 	 Acc: 0.731707 Size 0.000000 Proportion 0.000000
Event 657/926 	 Acc: 0.800000 Size 0.283333 Proportion 0.212500
Event 658/926 	 Acc: 0.866667 Size 0.650000 Proportion 0.443182
Event 659/926 	 Acc: 0.671642 Size 1.104478 Proportion 0.986667
Event 660/926 	 Acc: 0.630252 Size 2.529412 Proportion 2.104895
Event 661/926 	 Acc: 0.765432 Size 0.234568 Proportion 0.202128
Event 662/926 	 Acc: 0.767442 Size 0.046

Event 776/926 	 Acc: 0.771429 Size 0.000000 Proportion 0.000000
Event 777/926 	 Acc: 0.766234 Size 1.142857 Proportion 0.880000
Event 778/926 	 Acc: 0.762712 Size 0.745763 Proportion 0.556962
Event 779/926 	 Acc: 0.711538 Size 0.192308 Proportion 0.144928
Event 780/926 	 Acc: 0.756098 Size 0.073171 Proportion 0.069767
Event 781/926 	 Acc: 0.741176 Size 1.635294 Proportion 1.241071
Event 782/926 	 Acc: 0.603774 Size 3.006289 Proportion 2.463918
Event 783/926 	 Acc: 0.564103 Size 0.076923 Proportion 0.103448
Event 784/926 	 Acc: 0.803030 Size 0.787879 Proportion 0.597701
Event 785/926 	 Acc: 0.837838 Size 0.216216 Proportion 0.148148
Event 786/926 	 Acc: 0.875000 Size 0.000000 Proportion 0.000000
Event 787/926 	 Acc: 1.000000 Size 0.000000 Proportion 0.000000
Event 788/926 	 Acc: 0.704545 Size 0.068182 Proportion 0.058824
Event 789/926 	 Acc: 0.741379 Size 1.344828 Proportion 1.218750
Event 790/926 	 Acc: 0.805556 Size 0.138889 Proportion 0.096154
Event 791/926 	 Acc: 0.777778 Size 0.259

Event 905/926 	 Acc: 0.744444 Size 2.444444 Proportion 2.056075
Event 906/926 	 Acc: 0.702128 Size 0.085106 Proportion 0.074074
Event 907/926 	 Acc: 0.727273 Size 0.204545 Proportion 0.163636
Event 908/926 	 Acc: 0.820000 Size 0.380000 Proportion 0.316667
Event 909/926 	 Acc: 0.581699 Size 4.738562 Proportion 4.073034
Event 910/926 	 Acc: 0.772727 Size 0.090909 Proportion 0.100000
Event 911/926 	 Acc: 0.775510 Size 0.061224 Proportion 0.053571
Event 912/926 	 Acc: 0.655914 Size 0.870968 Proportion 0.764151
Event 913/926 	 Acc: 0.821429 Size 0.285714 Proportion 0.222222
Event 914/926 	 Acc: 0.895833 Size 0.041667 Proportion 0.031250
Event 915/926 	 Acc: 0.791209 Size 1.285714 Proportion 0.921260
Event 916/926 	 Acc: 0.543210 Size 2.827160 Proportion 2.544444
Event 917/926 	 Acc: 0.750000 Size 0.000000 Proportion 0.000000
Event 918/926 	 Acc: 0.686275 Size 0.607843 Proportion 0.534483
Event 919/926 	 Acc: 0.628099 Size 1.057851 Proportion 0.984615
Event 920/926 	 Acc: 0.729167 Size 0.270

In [22]:
X_classifier = []
Y_classifier = []

total_found = 0
total = 0
long_range = 300

for particle in hits_total:
    
    if (len(particle[5]) > 1) and (particle[0] != 11) and (particle[0] != -11):
        
        found_num = 0
        found_layers = np.zeros(6)
        # Extract only 12 hits, as some of them are really close
        X_final = {}
        Zone_final = {}
        actual = -1
        
        for i in range(0, len(particle[5])):
            if (actual != particle[8][i]):
                X_final[particle[8][i]] = []
                X_final[particle[8][i]].append(particle[5][i])
                actual = particle[8][i]
                Zone_final[particle[8][i]] = particle[12][i]
                found_num += 1
            else:
                X_final[actual].append(particle[5][i])
                
        if (found_num < 12):
            continue
        
        found_num = 0
        tmp_found = 0
        for station in range(0,3):
            
            # Compute the layer number based on station
            x0_layer = 0
            if (station == 1):
                x0_layer = 4
            elif station == 2:
                x0_layer = 8

            u_layer = x0_layer + 1
            v_layer = x0_layer + 2
            x1_layer = x0_layer + 3

            # Select both the first X position and UP/DOWN based on this
            first_hit = X_final[x0_layer][0]
            zone = Zone_final[x0_layer]
            
            found_layers[0] = first_hit

            predicted = (xlayer_model.predict(np.array([first_hit, x0_layer]).reshape(1,-1)))[0]
            mean_predicted = ((first_hit + predicted[0] + predicted[1] + predicted[2]) / 4)

            found = list(idx[particle[13]].intersection((predicted[2] - long_range, predicted[2] + long_range, x1_layer, x1_layer, zone, zone),objects='raw'))
            found_layer3 = sorted(found, key=lambda x: np.abs(x[0] - mean_predicted))[:12]
            for i in range(0,len(found_layer3)):
                if (found_layer3[i] in X_final[x1_layer]):
                    found_num += 1
                    found_layers[3] = found_layer3[i]
                    break

            found = list(idx[particle[13]].intersection((predicted[1] - long_range, predicted[1] + long_range, v_layer, v_layer, zone,zone),objects='raw'))
            found_layer2 = sorted(found, key=lambda x: np.abs(x[0] - mean_predicted))[:12]
            for i in range(0,len(found_layer2)):
                if (found_layer2[i] in X_final[v_layer]):
                    found_num += 1
                    found_layers[2] = found_layer2[i]
                    break

            found = list(idx[particle[13]].intersection((predicted[0] - long_range, predicted[0] + long_range, u_layer, u_layer,zone,zone),objects='raw'))
            found_layer1 = sorted(found, key=lambda x: np.abs(x[0] - mean_predicted))[:12]
            for i in range(0,len(found_layer1)):
                if (found_layer1[i] in X_final[u_layer]):
                    found_num += 1
                    found_layers[1] = found_layer1[i]
                    break

            if (found_num == 3):
                
                u_hit = random.choice(found_layer1)
                v_hit = random.choice(found_layer2)
                retries_x1 = 0
                while x1_hit == first_hit and retries_x1 != 4:
                    x1_hit = random.choice(found_layer3)
                    retries_x1 += 1
                
                if (first_hit == found_layers[3]):
                    continue
                
                found_layers[4] = predictHeight(first_hit, found_layers[3], found_layers[1], u_layer, x0_layer, x1_layer)
                found_layers[5] = predictHeight(first_hit, found_layers[3], found_layers[2], v_layer, x0_layer, x1_layer)

                if (retries_x1 < 4):
                    fake_u_height = predictHeight(first_hit, x1_hit, u_hit, u_layer, x0_layer, x1_layer)
                    fake_v_height = predictHeight(first_hit, x1_hit, v_hit, v_layer, x0_layer, x1_layer)
                    fake_track = np.array([first_hit, u_hit,v_hit,x1_hit, fake_u_height, fake_v_height])

                    true_track = found_layers
                    retries = 0

                    while (fake_track == true_track).all() and retries != 4:
                        u_hit = random.choice(found_layer1)
                        v_hit = random.choice(found_layer2)
                        retries_x1 = 0
                        while x1_hit == first_hit and retries_x1 != 4:
                            x1_hit = random.choice(found_layer3)
                            retries_x1 += 1
                        if (retries_x1 == 4):
                            retries = 4
                            break
                        fake_u_height = predictHeight(first_hit, x1_hit, u_hit, u_layer, x0_layer, x1_layer)
                        fake_v_height = predictHeight(first_hit, x1_hit, v_hit, v_layer, x0_layer, x1_layer)
                        fake_track = np.array([first_hit, u_hit, v_hit, x1_hit, fake_u_height, fake_v_height])
                        retries += 1

                    if (retries == 4):
                        X_classifier.append(true_track)
                        Y_classifier.append(1)
                    else:
                        X_classifier.append(true_track)
                        Y_classifier.append(1)
                        X_classifier.append(fake_track)
                        Y_classifier.append(0)
                else:
                    X_classifier.append(true_track)
                    Y_classifier.append(1)
                
                tmp_found += 1
                    
                
        if (tmp_found > 2):
            total_found += 1
        total += 1

print("Mean found %f" % (total_found/total))
print("Total %d" % total)

Mean found 0.000000
Total 69


In [26]:
track_classifier = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
track_classifier.add(Dense(12, activation='relu', input_dim=12))
track_classifier.add(Dense(8, activation='relu'))
track_classifier.add(Dense(1, activation='sigmoid'))
track_classifier.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [27]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X_classifier), np.array(Y_classifier))
print("X Train length: %d" % len(X_train))
print("X Test Length %d" % len(X_test))
print("Y Train length: %d" % len(y_train))
print("Y Test length: %d" % len(y_test))

X Train length: 138489
X Test Length 46164
Y Train length: 138489
Y Test length: 46164


In [28]:
track_classifier.fit(X_train, y_train, epochs=15, batch_size=20, verbose=1, validation_split=0.1)

Train on 124640 samples, validate on 13849 samples
Epoch 1/15
124640/124640 [==============================] - 9s 76us/sample - loss: 1.1587 - acc: 0.6614 - val_loss: 0.4921 - val_acc: 0.7052
Epoch 2/15
124640/124640 [==============================] - 10s 81us/sample - loss: 0.3688 - acc: 0.8561 - val_loss: 0.3000 - val_acc: 0.8864
Epoch 3/15
124640/124640 [==============================] - 10s 78us/sample - loss: 0.2345 - acc: 0.9109 - val_loss: 0.1918 - val_acc: 0.9281
Epoch 4/15
124640/124640 [==============================] - 10s 79us/sample - loss: 0.2158 - acc: 0.9205 - val_loss: 0.2198 - val_acc: 0.9295
Epoch 5/15
124640/124640 [==============================] - 9s 72us/sample - loss: 0.1930 - acc: 0.9305 - val_loss: 0.3826 - val_acc: 0.8550
Epoch 6/15
124640/124640 [==============================] - 8s 66us/sample - loss: 0.1887 - acc: 0.9319 - val_loss: 0.1744 - val_acc: 0.9399
Epoch 7/15
124640/124640 [==============================] - 8s 66us/sample - loss: 0.1813 - acc: 0.9

In [29]:
score, acc = track_classifier.evaluate(X_test, y_test, batch_size=25)
print('Test score:', score)
print('Test accuracy:', acc)

46164/46164 [==============================] - 1s 28us/sample - loss: 0.1584 - acc: 0.9438
Test score: 0.15835131678451683
Test accuracy: 0.94383067


In [30]:
track_classifier.save("./models/track_classifier_all_stations_v2.h5")

In [48]:
import datetime as dt
X_test = []

total_found = 0
total = 0
long_range = 300

hits_total = mc_hits[mc_hits[:,13] < event_threshold]

for particle in hits_total:
    
    if (len(particle[5]) > 1):
        
        found_bool = 0
        found_layers = np.zeros(3)
        # Extract only 12 hits, as some of them are really close
        zone = particle[12][0]
        X_final = {}
        actual = -1
        
        for i in range(0, len(particle[5])):
            if (actual != particle[8][i]):
                X_final[particle[8][i]] = []
                X_final[particle[8][i]].append(particle[5][i])
                actual = particle[8][i]
            else:
                X_final[actual].append(particle[5][i])
                
        if (not 0 in X_final):
            continue
            
        predicted = (xlayer_model.predict(np.array([X_final[0][0], 0]).reshape(1,-1)))[0]
        
        print(predicted)
        
        if (4 in X_final):
            found = list(idx[particle[13]].intersection((predicted[3] - long_range, predicted[3] + long_range, 4, 4,zone,zone),objects='raw'))
            found_layer4 = sorted(found, key=lambda x: np.abs(x - predicted[3]))[:10]
            found_bool += 1
        
        if (3 in X_final):
            found = list(idx[particle[13]].intersection((predicted[2] - long_range, predicted[2] + long_range, 3, 3, zone, zone),objects='raw'))
            found_layer3 = sorted(found, key=lambda x: np.abs(x - predicted[2]))[:10]
            found_bool += 1
            
        if (2 in X_final):
            found = list(idx[particle[13]].intersection((predicted[1] - long_range, predicted[1] + long_range, 2, 2, zone,zone),objects='raw'))
            found_layer2 = sorted(found, key=lambda x: np.abs(x - predicted[1]))[:10]
            found_bool += 1
        
        if (1 in X_final):
            found = list(idx[particle[13]].intersection((predicted[0] - long_range, predicted[0] + long_range, 1, 1,zone,zone),objects='raw'))
            found_layer1 = sorted(found, key=lambda x: np.abs(x - predicted[0]))[:10]
            found_bool += 1
            
        if (found_bool == 4):
            retries = 0
            
            valid_list = []
            max_val = 0
            for h4 in found_layer4:
                for h3 in found_layer3:
                    for h2 in found_layer2:
                        for h1 in found_layer1:
                            pred = (track_classifier.predict(np.array([X_final[0][0], h1, h2, h3, h4]).reshape(1,-1))[0])[0]
                            if pred > max_val:
                                valid = [X_final[0][0], h1, h2, h3, h4]
                                max_val = pred
            
            if valid[1] in X_final[1] and valid[2] in X_final[2] and valid[3] in X_final[3] and valid[4] in X_final[4]:
                total_found += 1
                
            print(total/354)
            print(total_found/354)
            #print(total_found/354)
            #print(total_val)
            #print('\n')
            
        total += 1

print("Mean found %f" % (total_found/total))
print("Total %d" % total)

[1344.3711 1331.2653 1353.7424 1449.1339]
0.0
0.0
[261.86023 264.366   268.9563  289.13846]
0.002824858757062147
0.0
[-244.05386 -283.24338 -305.46356 -349.80124]
0.005649717514124294
0.0
[139.9398  144.20387 146.77966 158.49117]
0.00847457627118644
0.0
[1401.4517 1387.5227 1410.9429 1510.3002]


KeyboardInterrupt: 